#Project 1 - Report 1 code

In [75]:
#Please have the 'playlists4000.txt' file in the same directory as this notebook.

fd = open("playlists4000.txt",mode='r',encoding='utf8',newline='\n')

raw = fd.read().splitlines()

hashtable = dict()

for line in raw:
    first, second = line.split(sep="::")
    hashtable.setdefault(first, []).append(second)

In [76]:
#Number of items
items = 0
for x in hashtable.values():
    items += len(x)
items

55994

In [77]:
#Number of transactions
len(hashtable.keys())

4000

In [78]:
#Longest transaction
longest = 0
for x in hashtable.values():
    if (len(x) > longest):
        longest = len(x)

longest

110

In [79]:
#Shortest transaction
shortest = longest
for x in hashtable.values():
    if(len(x) < shortest):
        shortest = len(x)
shortest

1

In [80]:
#Support counting

#Dictionary of 1-itemsets with support count

counts = dict()

for line in raw:
    first, second = line.split(sep="::")
    x = counts.setdefault(second,0)
    counts[second] = x+1

In [81]:
#Number of frequent 1-itemsets with support count threshold 400
freq = 0

for x in counts:
    if counts[x] > 40:
        freq += 1

freq

455

In [92]:
# minsup, minconf = 5, 0.5
minsup, minconf = 30, 0.7
# minsup, minconf = 60, 0.5
# minsup, minconf = 60, 0.5

In [93]:
# FP-Tree of 4-itemsets with support count
from collections import OrderedDict
fp = OrderedDict()

ncounts = dict()

for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        fp[i] = fp.get(i, OrderedDict())
        fp[i]["count"] = fp[i].get("count", 0) + 1

for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        if fp[i]["count"] < minsup:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            fp[i][j] = fp[i].get(j, OrderedDict())
            fp[i][j]["count"] = fp[i][j].get("count", 0) + 1
            
for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        if fp[i]["count"] < minsup:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            if fp[i][j]["count"] < minsup:
                continue
            for ki in range(ji+1, len(t)):
                k = int(t[ki])
                fp[i][j][k] = fp[i][j].get(k, OrderedDict())
                fp[i][j][k]["count"] = fp[i][j][k].get("count", 0) + 1

for t in hashtable.values():
    for ii in range(len(t)):
        i = int(t[ii])
        if fp[i]["count"] < minsup:
            continue
        for ji in range(ii+1, len(t)):
            j = int(t[ji])
            if fp[i][j]["count"] < minsup:
                continue
            for ki in range(ji+1, len(t)):
                k = int(t[ki])
                if fp[i][j][k]["count"] < minsup:
                    continue
                for li in range(ki+1, len(t)):
                    l = int(t[li])
                    fp[i][j][k][l] = fp[i][j][k].get(l, OrderedDict())
                    fp[i][j][k][l]["count"] = fp[i][j][k][l].get("count", 0) + 1
                
ctr = 0
for k, v in fp.items():
    ctr += v["count"]
fp["count"] = ctr


NameError: name 'li' is not defined

In [84]:
print(fp["count"])

55994


In [85]:
# Support counting

# Dictionary of 1-itemsets, 2-itemsets, and 3-itemsets with support count

ncounts = dict()
ncounts[1] = OrderedDict()
ncounts[2] = OrderedDict()
ncounts[3] = OrderedDict()
ncounts[4] = OrderedDict()

def dfs(node, s, len=0):
    if node["count"] < minsup:
        return
    if len >= 1:
        itemset = tuple(sorted(s))
        ncounts[len][itemset] = node["count"]
    if len == 4:
        return
    for k in node:
        if k != "count":
            dfs(node[k], s + [int(k)], len+1)

dfs(fp, [])

In [86]:
# Rule generation for 2-itemsets with minsup a and minconf b
rules = dict()
rules[2] = dict()
rules[3] = dict()
rules[2]["antecedent"] = list()
rules[2]["consequent"] = list()
rules[2]["support count"] = list()
rules[2]["confidence"] = list()


for k, v in ncounts[2].items():
    # print(", ".join(list(map(str, list(k)))), end="\t")
    # print("count: ", str(v))
    f, s = k
    n, d = v, ncounts[1][tuple([f])]
    conf = n / d
    if conf >= minconf:
        rules[2]["antecedent"].append(f)
        rules[2]["consequent"].append(s)
        rules[2]["support count"].append(n)
        rules[2]["confidence"].append(round(conf, 4))
        # rules[2][f"{f} -> {s}"] =  [round(conf, 4), n]
    d = ncounts[1][tuple([s])]
    conf = n / d
    if conf >= minconf:
        rules[2]["antecedent"].append(f)
        rules[2]["consequent"].append(s)
        rules[2]["support count"].append(n)
        rules[2]["confidence"].append(round(conf, 4))
        # rules[2][f"{f} -> {s}"] = [round(conf, 4), n]

# for k, v in rules[2].items():
#     print(k, end="\t\t")
#     print("support count: ", str(v[1]), end="\t\t")
#     print("confidence: ", str(v[0])) 

In [87]:
# Rule generation for 3-itemsets with minsup a and minconf b
rules[3]["antecedent"] = list()
rules[3]["consequent"] = list()
rules[3]["support count"] = list()
rules[3]["confidence"] = list()
seen = set()

def arrange2(three: tuple, prevant: tuple):
    f, m, e = three
    pf, ps = prevant
    diff = list(set([f, m, e]).difference(set([pf, ps])))[0]
    ant, cons =  tuple([pf]), tuple(sorted([ps, diff]))
    n, d = ncounts[3][three], ncounts[1][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in seen:
        seen.add(rulestr)
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(conf)
        # rules[3][rulestr] = [round(conf, 4), n]
    ant, cons =  tuple([ps]), tuple(sorted([pf, diff]))
    n, d = ncounts[3][three], ncounts[1][ant]
    conf = n / d
    sant = ", ".join(list(map(str, list(ant))))
    scons = ", ".join(list(map(str, list(cons))))
    rulestr = f"{sant}\t--->\t{scons}"
    if conf >= minconf and rulestr not in rules[3]:
        seen.add(rulestr)
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(conf)
        # rules[3][rulestr] = [round(conf, 4), n]
    

def arrange(three: tuple):
    f, m, e = three
    ant = tuple(sorted([m, e]))
    cons  = tuple([f])
    n, d = ncounts[3][three], ncounts[2][ant]
    conf = n / d
    if conf >= minconf:
        sant = ", ".join(list(map(str, list(ant))))
        scons = ", ".join(list(map(str, list(cons))))
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(conf)
        rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        arrange2(three, ant)
    ant, cons = tuple(sorted([f, e])), tuple([m])
    n, d = ncounts[3][three], ncounts[2][ant]
    conf = n / d
    if conf >= minconf:
        sant = ", ".join(list(map(str, list(ant))))
        scons = ", ".join(list(map(str, list(cons))))
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(conf)
        # rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        arrange2(three, ant)
    ant, cons = tuple(sorted([f, m])), tuple([e])
    n, d = ncounts[3][three], ncounts[2][ant]
    conf = n / d
    if conf >= minconf:
        sant = ", ".join(list(map(str, list(ant))))
        scons = ", ".join(list(map(str, list(cons))))
        rules[3]["antecedent"].append(sant)
        rules[3]["consequent"].append(scons)
        rules[3]["support count"].append(n)
        rules[3]["confidence"].append(conf)
        # rules[3][f"{sant}\t--->\t{scons}"] = [round(conf, 4), n]
        arrange2(three, ant)
    
    
for k, v in ncounts[3].items(): 
    arrange(k)

# for k, v in rules[3].items():
#     print(k, end="\t\t")
#     print("support count: ", str(v[1]), end="\t\t")
#     print("confidence: ", str(v[0]))   

In [88]:
#Dataframe for 2-itemset rules
import pandas as pd

two_itemset_rules = pd.DataFrame(rules[2])

In [89]:
two_itemset_rules.head()

,antecedent,consequent,support count,confidence
0,27,167,68,0.5271
1,27,297,65,0.5039
2,37,49,75,0.5068
3,51,804,61,0.5922
4,51,804,61,0.6778


In [90]:
#Dataframe for 3-itemset rules
import pandas as pd

three_itemset_rules = pd.DataFrame(rules[3])

In [91]:
three_itemset_rules.head()

,antecedent,consequent,support count,confidence


Turns out there are no 1-temsets with a support of 400

In [70]:
#Highest support for a 1-itemset
max(counts.values())

194

In [71]:
import pandas as pd

df_counts = pd.DataFrame.from_dict(counts, orient='index')

df_counts = df_counts.set_axis(['support'],axis=1)

df_counts.head()

,support
0,132
1,106
2,123
3,77
4,86


In [54]:
plot_df = df_counts.loc[df_counts['support'] > 50]

plot_df = plot_df.sort_values(by=['support']).reset_index()

plot_df = plot_df.rename(columns={'index' : 'Items','support':'Support'})

plot_df

,Items,Support
0,541,51
1,805,51
2,315,51
3,322,51
4,439,51
...,...,...
329,320,172
330,167,172
331,297,186
332,203,187


In [72]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(num=1,figsize=(20,15), dpi=100)
plt.bar(plot_df['Items'], plot_df['Support'])
plt.xticks(np.arange(0, 333, 10))
plt.xlabel('Item')
plt.ylabel('Support')
plt.title('Support counts in increasing order')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'